**conda env-** faus_dl *with python 3 kernel*

## List of Datasets:

  * DISFA
  * CK+
  * UNBC Shoulder Pain Dataset
  * AM-FED (Affectiva)
  * FERA 2015 and 2017 Challenge Datasets

Importing libraries:

In [19]:
import glob
import os
import sys
import pandas as pd
import numpy as np

### **DISFA**

http://mohammadmahoor.com/wp-content/uploads/2017/06/DiSFA_Paper_andAppendix_Final_OneColumn1-1.pdf

* 16.5 GB
* 12 FAUs intensity
* 27 subjects (15 male, 12 female)
* 130,000 frames -each video has 4845 frames @ 20 fps
* Page 5 of paper has the distribution of occurrence of FAUs, each FAU occurs in atleast 5000 frames

#### Contains videos from the left and the right cameras in avi format, and also FAU labels.
**Videos Name formatting:**

*Videos_LeftCamera/{L,l}eftVideoSN001_{c,C}omp.avi* **or** *Videos_RightCamera/{R,r}ightVideoSN001_{c,C}omp.avi*

**FAU label formatting:**

*ActionUnit_Labels/SN001/SN001_au1.txt*

inside this file, each row is like:   *frame_no,intensity{0,1,2,3,4,5}*

In [3]:
DISFA_path='/media/amogh/Stuff/CMU/datasets/DISFA_data/'

In [2]:
% ls {DISFA_path}

ls: cannot access {DISFA_path}: No such file or directory


In [20]:
DISFA_AU_path=DISFA_path+'ActionUnit_Labels/'
print(DISFA_AU_path)
Videos_right_path=DISFA_path+'Videos_RightCamera/'
print(Videos_right_path)

/media/amogh/Stuff/CMU/datasets/DISFA_data/ActionUnit_Labels/
/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/


**Loading FAUs**

**Extracting frames**

In [21]:
os.getcwd()

'/home/amogh/cmu/notebooks'

In [22]:
f'{Videos_right_path}*.avi'

'/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/*.avi'

In [23]:
glob.glob(f'{Videos_right_path}*.avi')[0].split('/')[-1].split('_')[0]

'RightVideoSN013'

Create folders, raw code to avoid overwrite

View files in Videos_right_path:

In [24]:
glob.glob(f'{Videos_right_path}*')

['/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN013_comp.avi',
 '/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN001',
 '/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN001_comp.avi',
 '/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN002',
 '/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN002_comp.avi',
 '/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN003',
 '/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN003_comp.avi',
 '/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN004',
 '/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN004_comp.avi',
 '/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN005',
 '/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN005_comp.avi',
 '/media/amogh/Stuff/CMU/datasets/DISFA_data

In [25]:
for path in glob.glob(f'{Videos_right_path}*.avi'):
    print(path)


/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN013_comp.avi
/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN001_comp.avi
/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN002_comp.avi
/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN003_comp.avi
/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN004_comp.avi
/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN005_comp.avi
/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN006_comp.avi
/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN007_comp.avi
/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN008_comp.avi
/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN009_comp.avi
/media/amogh/Stuff/CMU/datasets/DISFA_data/Videos_RightCamera/RightVideoSN010_comp.avi
/media/amogh/Stuff/CMU/datasets/DISFA_data/

### CK+

### UNBC Shoulder Pain Dataset

### AM-FED(Affectiva)

## **FERA 2015 and 2017 Challenges**

### **BP4D**

In [4]:
BP4D_base_path='/home/amogh/cmu/dataset/BP4D/'


**Loading FAUs:**
<br>
AUCoding has csv files for each sequence. Eg- F001_T1.csv
<br>
Each csv file has a row corresponding to each frame, and corresponding columns 1-27 represent FAUs.
<br>
Occurrence codes: 0 for absent, 1 for present, or 9 for missing data (unknown).



In [5]:
BP4D_AU_path=BP4D_base_path+'AUCoding/AUCoding/'

#### **BP4D example functions**

In [6]:
example_subject='F001'
example_sequence='T1'
example_file=f'{example_subject}_{example_sequence}.csv'
example_file

'F001_T1.csv'

In [7]:
df_example=pd.read_csv(BP4D_AU_path+example_file)
df_example.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,2440,0,0,9,0,0,0,0,9,0,...,9,9,9,9,9,9,9,9,9,9
1,2441,0,0,9,0,0,0,0,9,0,...,9,9,9,9,9,9,9,9,9,9
2,2442,0,0,9,0,0,0,0,9,0,...,9,9,9,9,9,9,9,9,9,9
3,2443,0,0,9,0,0,0,0,9,0,...,9,9,9,9,9,9,9,9,9,9
4,2444,0,0,9,0,0,0,0,9,0,...,9,9,9,9,9,9,9,9,9,9


In [8]:
frame_numbers=df_example['0']
frame_numbers

0      2440
1      2441
2      2442
3      2443
4      2444
5      2445
6      2446
7      2447
8      2448
9      2449
10     2450
11     2451
12     2452
13     2453
14     2454
15     2455
16     2456
17     2457
18     2458
19     2459
20     2460
21     2461
22     2462
23     2463
24     2464
25     2465
26     2466
27     2467
28     2468
29     2469
       ... 
523    2963
524    2964
525    2965
526    2966
527    2967
528    2968
529    2969
530    2970
531    2971
532    2972
533    2973
534    2974
535    2975
536    2976
537    2977
538    2978
539    2979
540    2980
541    2981
542    2982
543    2983
544    2984
545    2985
546    2986
547    2987
548    2988
549    2989
550    2990
551    2991
552    2992
Name: 0, Length: 553, dtype: int64

**Seeing the occurrence (0(not present)/1(present)/9(unlabelled)) of an FAU number for a frame**;  *nth* AU value is the *nth* column for that frame

In [9]:
frame_no=2441
fau_no=3
df2=(df_example.loc[df_example['0']==frame_no]).iloc[:,fau_no]
df2
# df_example.loc[(frame_no),str(fau_no)]

1    9
Name: 3, dtype: int64

Seeing the **number of times each FAU occurs(1), does not occur(0), isn't labelled(9)**; syntax- df_example.iloc[:,0:28] returns the column for 27 FAUs, apply the value_counts function to each column, which gives the frequency of all values that occur in it. iloc at end is used to just see '0','1','9'. 

In [10]:
df_example.iloc[:,0:28].apply(pd.Series.value_counts).iloc[:3,:]

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,NaN,233.0,415.0,NaN,527.0,432.0,263.0,553.0,NaN,553.0,...,553.0,553.0,537.0,NaN,553.0,553.0,553.0,NaN,NaN,496.0
1,NaN,320.0,138.0,NaN,26.0,121.0,290.0,NaN,NaN,NaN,...,NaN,NaN,16.0,NaN,NaN,NaN,NaN,NaN,NaN,57.0
9,NaN,NaN,NaN,553.0,NaN,NaN,NaN,NaN,553.0,NaN,...,NaN,NaN,NaN,553.0,NaN,NaN,NaN,553.0,553.0,NaN


In [17]:
def label_getter(subj_req,seq_req,frame_req):
    au_file_reqd=BP4D_AU_path+f'{subj_req}_{seq_req}.csv'
    print (au_file_reqd)
    df_reqd=pd.read_csv(au_file_reqd)
    list_of_faus=list(df_reqd.loc[df_example['0']==frame_req].iloc[0]) #list: [frame_no, fau1, fau2, fau3....]
    #choose FAUS for which you want the labels.
    list_mask=[1,2,4,6,7,10,12,14,15,17,23]
    list_final=[list_of_faus[i] for i in list_mask]
    return list_final

#running on an example.
label_getter('F001','T1',2440)

/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/F001_T1.csv


[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]

**Analysing occurrence of FAUs**

In [23]:
list_of_AU_files=glob.glob(f'{BP4D_AU_path}/*')
list_of_AU_files

['/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/M005_T5.csv',
 '/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/M012_T2.csv',
 '/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/M010_T1.csv',
 '/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/F015_T3.csv',
 '/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/F021_T6.csv',
 '/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/F005_T3.csv',
 '/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/M013_T2.csv',
 '/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/F004_T2.csv',
 '/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/F008_T5.csv',
 '/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/M002_T7.csv',
 '/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/F014_T1.csv',
 '/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/M007_T3.csv',
 '/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/M018_T6.csv',
 '/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/M003_T2.csv',
 '/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/F019_T2.csv',
 '/home/amogh/cmu/dataset/BP4D/AUCoding/AUCoding/F017_T

Let's try to create csv files for each relevant FAU.<br>
Rows - frameNo_seqNo <br>
Columns - fauNo

FAUs for which you want a csv file to be created

In [142]:
relevant_fau=[1,2,4,5,7,12,25,26,43]

intialising FAU dictionaries for above FAUS

In [153]:
all_dict={}
for fau in relevant_fau:
    all_dict[f"dict_{fau}"]={}
all_dict

{'dict_1': {},
 'dict_2': {},
 'dict_4': {},
 'dict_5': {},
 'dict_7': {},
 'dict_12': {},
 'dict_25': {},
 'dict_26': {},
 'dict_43': {}}

for each csv:

In [165]:
for csv_file in list_of_AU_files:
    csv_df=pd.read_csv(csv_file)
    csv_basename=os.path.basename(csv_file)
    user_seq=os.path.splitext(csv_basename)[0]
    #1st column is for frame_no, returns the occurrence dataframe for relevant FAUs
    csv_relevant_df=csv_df.iloc[:,np.insert(relevant_fau,0,0)] 
    #adding to dictionary for each FAU:
    for fau in relevant_fau:
        occurrence_frames=(csv_relevant_df['0'][csv_relevant_df[str(fau)]==1]).values #list of all frames with au occurence    
        all_dict[f'dict_{fau}'][f'{user_seq}']=occurrence_frames

In [166]:
sample_df=pd.DataFrame(dict([(k,pd.Series(v)) for k,v in (all_dict['dict_5']).items()]))
sample_df

,F001_T1,F001_T2,F001_T3,F001_T4,F001_T5,F001_T6,F001_T7,F001_T8,F002_T1,F002_T2,...,M017_T7,M017_T8,M018_T1,M018_T2,M018_T3,M018_T4,M018_T5,M018_T6,M018_T7,M018_T8
0,2466.0,1096.0,135.0,671.0,NaN,242.0,NaN,NaN,42.0,443.0,...,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,573.0,NaN
1,2467.0,1097.0,136.0,672.0,NaN,243.0,NaN,NaN,43.0,444.0,...,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,574.0,NaN
2,2468.0,1098.0,137.0,673.0,NaN,244.0,NaN,NaN,44.0,445.0,...,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,575.0,NaN
3,2469.0,1099.0,138.0,674.0,NaN,245.0,NaN,NaN,45.0,446.0,...,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,613.0,NaN
4,2470.0,1100.0,139.0,675.0,NaN,246.0,NaN,NaN,46.0,447.0,...,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN,614.0,NaN
5,2471.0,1101.0,140.0,676.0,NaN,247.0,NaN,NaN,47.0,448.0,...,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,615.0,NaN
6,2472.0,1102.0,141.0,677.0,NaN,248.0,NaN,NaN,48.0,449.0,...,NaN,NaN,NaN,NaN,16.0,NaN,NaN,NaN,616.0,NaN
7,2473.0,1103.0,142.0,678.0,NaN,NaN,NaN,NaN,49.0,450.0,...,NaN,NaN,NaN,NaN,74.0,NaN,NaN,NaN,617.0,NaN
8,2474.0,1104.0,143.0,679.0,NaN,NaN,NaN,NaN,50.0,451.0,...,NaN,NaN,NaN,NaN,75.0,NaN,NaN,NaN,618.0,NaN
9,2475.0,1105.0,144.0,680.0,NaN,NaN,NaN,NaN,51.0,452.0,...,NaN,NaN,NaN,NaN,76.0,NaN,NaN,NaN,619.0,NaN


**Loading images:**
<br>
BP4D training has folders as *subject/sequence/frame_no.jpg*

In [126]:
BP4D_training_folder=BP4D_base_path+'BP4D-training/'

In [136]:
!ls $BP4D_training_folder

F001


### **SEMAINE**

* 150 participants' recordings; total 959 conversations, ~5 minutes each
* FACs annotation in 181 frames
